# load packages

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

# read in input files

## scores

In [ ]:
avg_gene = pd.read_csv('pathway_score/rosmap/avg_pathways/AOU_ALL.UKBB.metasoft.gene_score.ROSMAP.RNAseq.methylation.somoscan_proteomics.MSBB.RNAseq.methylation.tmt_proteomics.ADSP.gene_average.pathway_scores.standard_scaled.go.keep_quest_comb.txt',
                       sep = '\t')
print(avg_gene.shape)
avg_gene.head()

In [ ]:
avg_pathway = pd.read_csv('pathway_score/rosmap/avg_pathways/AOU_ALL.UKBB.metasoft.gene_score.ROSMAP.RNAseq.methylation.somoscan_proteomics.MSBB.RNAseq.methylation.tmt_proteomics.ADSP.pathway_average.pathway_scores.standard_scaled.go.keep_quest_comb.txt',
                          sep = '\t')
print(avg_pathway.shape)
avg_pathway.head()

## pheno

In [ ]:
adsp_pheno = pd.read_csv('/project/ritchie/projects/ADSP_Projects/QC_ADSPv11/ADSPphenotype_forAnalysis.txt',
                         sep = '\t')
print(len(adsp_pheno.index))
adsp_pheno.head()

## id map

In [ ]:
id_map = pd.read_csv('pathway_score/id_map/ADSP.ROSMAP.MSBB.id_map.txt',
                      sep = '\t')
id_map.head()

## keep quest comb

In [ ]:
keep_quest_comb = pd.read_csv('adsp_filt_phenos/ADSPIntegratedPhenotypes_DS_2023.08.08.keep_quest_comb_cohorts_samples.csv',
                              header = None)
print(len(keep_quest_comb.index))
keep_quest_comb.head()

## pathway filter

In [ ]:
go_bio_path = pd.read_csv('pathway_score/pathway_annotation/go/AD_KMI.ADSP.ROSMAP.all_omics.MSBB.all_omics.GRCh38.113.refseq.exp_validated.jaccard_similarity_less_80.go_bio.gene_to_pathway.no_duplicates.pathway_mapping.txt',
                          sep = '\t')
print(len(go_bio_path.index))
print(len(go_bio_path['PATHWAY_ID'].unique()))
go_bio_path.head()

# filter pathway scores

In [ ]:
avg_pathway_filt = avg_pathway[avg_pathway['PATHWAY_ID'].isin(go_bio_path['PATHWAY_ID'])]
print(len(avg_pathway.index))
print(len(avg_pathway_filt.index))

In [ ]:
avg_gene_filt = avg_gene[avg_gene['PATHWAY_ID'].isin(go_bio_path['PATHWAY_ID'])]
print(len(avg_gene.index))
print(len(avg_gene_filt.index))

# transpose

In [ ]:
avg_pathway_transpose = avg_pathway_filt.set_index('PATHWAY_ID')
avg_pathway_transpose = avg_pathway_transpose.transpose()
avg_pathway_transpose.insert(0, 'ID', avg_pathway_transpose.index)
print(avg_pathway_transpose.shape)
avg_pathway_transpose.head()

In [ ]:
avg_gene_transpose = avg_gene_filt.set_index('PATHWAY_ID')
avg_gene_transpose = avg_gene_transpose.transpose()
avg_gene_transpose.insert(0, 'ID', avg_gene_transpose.index)
print(avg_gene_transpose.shape)
avg_gene_transpose.head()

# clean adsp pheno

In [ ]:
adsp_pheno = adsp_pheno.rename(columns = {'IID' : 'SampleID'})
adsp_pheno_id = id_map.merge(adsp_pheno, on = 'SampleID', how = 'inner')
adsp_pheno_id = adsp_pheno_id[adsp_pheno_id['SampleID'].isin(keep_quest_comb[0])]
adsp_pheno_id = adsp_pheno_id[['CommonID', 'DX_harmonized', 'Age_harmonized', 'Sex', 'PC1', 'PC2', 'PC3', 'PC4']]
adsp_pheno_id.rename(columns = {'CommonID' : 'ID',
                                'DX_harmonized' : 'AD',
                                'Age_harmonized' : 'Age'}, inplace = True)
print(len(adsp_pheno_id.index))
adsp_pheno_id.dropna(inplace = True)
print(len(adsp_pheno_id.index))
adsp_pheno_id.head()

In [ ]:
id_map_keep_quest_comb = id_map[id_map['SampleID'].isin(keep_quest_comb[0])]
print(len(id_map_keep_quest_comb.index))
id_map_keep_quest_comb.to_csv('pathway_score/id_map/ADSP.ROSMAP.MSBB.keep_quest_comb.id_map.txt',
                              sep = '\t',
                              na_rep = 'NaN')

# merge

In [ ]:
avg_gene_merge = adsp_pheno_id.merge(avg_gene_transpose, on = 'ID')
print(avg_gene_merge.shape)
avg_gene_merge.head()

In [ ]:
avg_pathway_merge = adsp_pheno_id.merge(avg_pathway_transpose, on = 'ID')
print(avg_pathway_merge.shape)
avg_pathway_merge.head()

# export

In [ ]:
avg_gene_merge.to_csv('ML/statistical_models/input/AOU_ALL.UKBB.metasoft.gene_score.ROSMAP.RNAseq.methylation.somoscan_proteomics.MSBB.RNAseq.methylation.tmt_proteomics.ADSP.gene_average.pathway_scores.standard_scaled.go.keep_quest_comb.covariates.statistical_models_input.txt',
                      sep = '\t',
                      index = None,
                      na_rep = 'NaN')

In [ ]:
avg_pathway_merge.to_csv('ML/statistical_models/input/AOU_ALL.UKBB.metasoft.gene_score.ROSMAP.RNAseq.methylation.somoscan_proteomics.MSBB.RNAseq.methylation.tmt_proteomics.ADSP.pathway_average.pathway_scores.standard_scaled.go.keep_quest_comb.covariates.statistical_models_input.txt',
                         sep = '\t',
                         index = None,
                         na_rep = 'NaN')